# TP4 - Flux optiques - visualisation et classification

Dans ce TP nous travaillerons avec un sous-ensemble de vidéos du corpus `UCF Sports` que vous pouvez télécharger depuis la page Moodle du cours.

Le fichier videos_samples.txt contient la liste des vidéos du corpus, ainsi que l'étiquette correspondante à chaque vidéo.

Ci-dessous vous pouvez trouver quelques fonctions utiles pour la suite du TP.

In [1]:
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt

def bgr2grayscale_numpy(img):
    return .0722*img[:,:,0] + .7152*img[:,:,1] + .2126*img[:,:,2]
    
def read_file_list(input_file):
    frame_list, label_list = [], []
    with open(input_file) as f:
        for l in f:
            l = l.strip().split(' ')
            frame_list.append(l[0])
            label_list.append(l[1])
    return np.array(frame_list), np.array(label_list)

def read_video(video_file):
    capture = cv2.VideoCapture(video_file)
    frames = []
    ok, frame = capture.read()
    if not(ok):
        print("empty file"+video_file)
    while ok:
        frames.append(frame[...,::-1]) # let's convert frames to RGB directly
        ok, frame = capture.read()
    return np.array(frames)

def optical_flow_farneback(previous_frame, next_frame):
    return cv2.calcOpticalFlowFarneback(previous_frame, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

## Caractérisation d'une vidéo au moyen des descripteurs extraits depuis chaque image de celle-ci

Nous pouvons traiter une vidéo comme une séquence d'images et, par conséquent, nous pouvons voir la description de la vidéo comme étant constituée des descripteurs extraits à partir des images la composant.

Dans la suite, nous nous intéressons aux descripteurs globaux tels que les histogrammes.

**Q1/** Ecrire un script python qui calcule les histogrammes couleur pour chaque image d'une vidéo.

In [4]:
def color_histogram(im, bins_per_channel=8):
    """Compute a normalized joint color histogram.

    Args:
        im: Color image as numpy array of shape (height, width, 3) and dtype uint8.
        bins_per_channel: Number of bins per channel after quantization.

    Returns:
        Normalized color histogram as numpy array of shape (bins_per_channel**3,)
        and dtype float32.
    """
    im = im.copy()

    # Quantize image
    bin_width = 256.0 / bins_per_channel
    im = (im / bin_width).astype(np.uint32)

    # Flatten color space
    im = im[..., 0] * bins_per_channel**2 + im[..., 1] * bins_per_channel + im[..., 2]

    # Compute and normalize histogram
    histogram = np.zeros((bins_per_channel**3,), dtype=np.float32)
    colors, counts = np.unique(im, return_counts=True)
    histogram[colors] = counts
    histogram = histogram / np.linalg.norm(histogram, ord=1)

    return histogram

In [40]:

import tensorflow as tf
def hist_video(video_path : str):
    hist = []
    images = read_video(video_path)
    for frame in images:
        tf.image.resize(frame, (128, 128))
        hist.append(color_histogram(frame))
    return np.array(hist)

**Q2/** Ecrire un script python qui calcule un histogramme moyen pour une vidéo, à partir des histogrammes couleur de chaque image la composant.

In [41]:
def hist_avg(video_path : str):
    hist = hist_video(video_path)
    return np.mean(hist, axis=0)

**Q3/** Si l'on considère l'histogramme moyen comme un descripteur pour une vidéo, mettez en place un protocole de classification sur les vidéos de la base `UCF Sports`. L'évaluation se fera en utilisant un processus de cross validation en 4 folds et un classifier de votre choix.

Veuillez utiliser les configurations suivantes :

a) utilisez que les données des classes `Diving-Side` / `Golf-Swing-Front`

b) utilisez que les données des classes `Kicking-Front` / `Golf-Swing-Front`

c) utilisez toutes les classes 

Reporter et discuter les résultats obtenus.

In [42]:
from pathlib import Path

def load_video_dataset(file_path):
    dataset = []

    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            video_path, label = line.split()
            dataset.append({
                "video_path": video_path,
                "annotation": label
            })

    return dataset



In [67]:
import pandas as pd
dataset  = load_video_dataset("/Users/ainazeaze/etude/amvo/amvo/ucf_sports_subset5/videos.files")

filtered_dataset = [
    item for item in dataset
    if item["annotation"] in ["Diving-Side", "Golf-Swing-Front"]
]

X = []
y = []
for item in filtered_dataset:
    hist = hist_avg("/Users/ainazeaze/etude/amvo/amvo/ucf_sports_subset5/"+item["video_path"])
    X.append(hist)
    y.append(0 if item["annotation"] == "Diving-Side" else 1)


In [68]:

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X, y)



,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",42
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`multi

In [69]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
    clf, X, y, cv=4, scoring='accuracy')
scores

array([0.33333333, 0.33333333, 1.        , 1.        ])

**Q4/** Maintenant, nous allons considérer que chaque vidéo est constitués des quatres parties distinctes. Chaque partie contient un quart des images comme suit : `partie1=video[:n/4], partie2=video[n/4+1:n/2], partie3=video[2*n/4+1:3*n/4], partie4=video[3*n/4+1:]`. Désormais, nous souhaitons que le descripteur d'une vidéo soit composé de 4 histogrammes (un pour chaque partie de la vidéo). Ecrire un script python qui calcule ce nouveau descripteur.

In [76]:
def hist_avg_4p(video_path : str):
    desc = []
    images = read_video(video_path)
    n = len(images)
    desc.append(color_histogram(tf.image.resize(images[:n//4], (128, 128))))
    desc.append(color_histogram(tf.image.resize(images[n//4+1:n//2], (128,128))))
    desc.append(color_histogram(tf.image.resize(images[2*n//4+1:3*n//4], (128, 128))))
    desc.append(color_histogram(tf.image.resize(images[3*n//4+1:], (128,128))))
    return np.array(desc)

**Q5/** Reprendre le protocole de la **Q3** mettez en place un protocole de classification sur les vidéos de la base `UCF Sports`. L'évaluation se fera en utilisant un processus de cross validation en 4 folds et un classifier de votre choix.

Veuillez reutiliser les mêmes configurations qu'en **Q3**. Reporter et discuter les résultats obtenus en insistant sur les avantages/désavantages rapportés par ce découpage de la vidéo. 

In [77]:
dataset  = load_video_dataset("/Users/ainazeaze/etude/amvo/amvo/ucf_sports_subset5/videos.files")

filtered_dataset = [
    item for item in dataset
    if item["annotation"] in ["Diving-Side", "Golf-Swing-Front"]
]

X = []
y = []
for item in filtered_dataset:
    hist = hist_avg_4p("/Users/ainazeaze/etude/amvo/amvo/ucf_sports_subset5/"+item["video_path"])
    X.append(hist)
    y.append(0 if item["annotation"] == "Diving-Side" else 1)


clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X, y)

scores = cross_val_score(
    clf, X, y, cv=4, scoring='accuracy')
scores

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'copy'

## Visualiser les flux optiques (en tant qu'images RGB)

Le flux optique permet de rendre compte du mouvement perçu dans la vidéo.

Afin de s'affranchir de la spécificité de la texture/couleur des objets en mouvement, il est intéressant d'utiliser directement l'information de mouvement. Un objet rouge et un objet vert se déplaçant de la même manière, auront des histogrammes de couleurs différentes mais partagerons les mêmes propriétés de mouvement.

En partant de cette observation, nous essayerons d'exploiter l'information du mouvement dans le cadre de la classification d'actions, en nous appuyant cette fois-ci sur l'information de mouvement.

**Q6/** Ecrire un script python qui calcule les flux optiques entre chaque deux images successives pour une vidéo.

Vous pouvez utiliser la fonction `calcOpticalFlowFarneback` avec les paramètres ci-dessous:
```
cv2.calcOpticalFlowFarneback(previous_frame, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
```

N'oubliez pas que vous devez procéder à une conversion des images en niveaux de gris à priori. Vous pouvez utiliser la fonction `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) ` pour cela.

**Q7/** Pour visualiser les flux optiques, nous passons par une transformation des valeurs (dx,dy) associées à un pixel à une représentation HSV où :
    * le canal `H` à la direction du flux `atan2(dy,dx)` et
    * le canal `V` correspond à la magnitude du flux (`la norme 2 du vecteur (dx,dy)`) normée elle-même sur l'intervalle `[0..1]`
    

```
flow_hsv[...,0] = np.arctan2(flow[...,1], flow[...,0])/np.pi*180. + 180.
flow_hsv[...,1] = 1
flow_hsv[...,2] = np.linalg.norm(flow, axis=2, ord=2)
flow_hsv[...,2] = (flow_hsv[...,2] - np.min(flow_hsv[...,2])) / (np.max(flow_hsv[...,2]) - np.min(flow_hsv[...,2]))
```

Selon que vous traitez l’ensemble des trames de flux d’une vidéo ou une trame de flux à la fois, vous devez créer le tensor `flow_hsv` de manière convenable au préalable :

```
#une trame à la fois
flow_hsv = np.empty((flow.shape[0],flow.shape[1],3), float32)

#les trames d’une video à la fois 
flows_hsv = np.empty((flow.shape[0],flow.shape[1],flow.shape[2],3), float32)
#dans ce cas, vous devez aussi remplacer axis=2, par axis=3 lors du calcul des normes
```

L’intérêt de transformer l’ensemble des trames à la fois et d’obtenir une normalisation des flux qui tient compte de l’étendue complète des magnitudes observées sur l’intégralité de la vidéo. Sans cela, les faibles magnitudes présentes sur les trames comportant peu de mouvements seront perçues comme fortes lors de la visualisation. 

Nous procédons ensuite à une conversion de `HSV` vers `RGB` pour visualiser les flux ainsi obtenus. La conversion en `RGB` va générer des intensités comprises entre `[0..1]` sur chaque canal.

Ecrivez une fonction qui permet de générer et sauvegarder les flux optiques sous formes d’images `RGB` pour une vidéo. Lors de la sauvegarde des images `RGB` pensez à multiplier par 255 (car valeurs comprises entre `[0..1]`) et de convertir en entiers (`int`) les valeurs contenues dans les tenseurs. Vous pouvez utiliser la fonction `np.astype` pour convertir tous les tenseurs en entiers.

Pensez à utiliser le dossier `/local` pour réaliser les sauvegardes de vos données.

## Vidéos - flux optiques comme orientations et magnitudes

Nous pouvons aussi exploiter les flux optiques directement sans passer par une représentation `RGB` de ceux-ci.

Nous considérons chaque point du flux comme un vecteur défini par son orientation et sa magnitude.

L'espace de représentation des orientations est borné et s'étale entre `0°` et `359°`.

En revanche, les magnitudes peuvent avoir des plages de représentation très larges, de part la vitesse de réalisation des actions, ou de part, les erreurs de mesure.

Afin de pouvoir construire des histogrammes qui traitent de l'orientation et de la magnitude conjointement, nous pouvons nous appuyer sur les valeurs min et max des magnitudes observées au sein du corpus de données.

**Q8/** Ecrivez une fonction qui transforme un flux optique `(dx,dy)` dans sa représentation ``orientation, magnitude)` en vous servant des règles suivantes:
```
flow_mo[...,0] = (np.arctan2(flow[...,1], flow[...,0])/np.pi*180. + 180.).astype(int)
flow_mo[...,1] = np.linalg.norm(flow, axis=2, ord=2).astype(int)
```
Nous employons ici une conversion vers des `np.array` avec `dtype=int` afin de faciliter la construction d’histogrammes par la suite.

**Q9/** Ecrivez une fonction qui extrait l'ensemble de magnitudes observées sur la première vidéo de chaque classe de mouvement et sauvegardez-les dans un fichier.

**Q10/** 
**a)** Ecrivez une fonction qui calcule un histogramme à 32 bins de ces magnitudes et visualisez-le afin d'identifier un seuil raisonnable pour les magnitudes apparaissant très rarement afin d'éliminer autant que possible les outliers. Les outliers corresponds souvent à des mesures abérantes causées par les erreurs se glissant lorsque les hypothèse de calcul de flux ne sont pas respectées. Ces magnitudes apparaîssent peu de fois par rapport aux magnitudes cohérentes.

Vous pourriez itérer plusieurs fois en construisant des histogramme à 32 bins sur des plages de moins en moins étendue, car il se peut que lors de la première itération la plage de magnitudes soit très large pour analyser convenablement la distribution des magnitudes coherentes. 

**b)** Vous pourriez ensuite essayer de filtrer également les magnitudes trop petites, ne correspondant à des véritables mouvements. En construisant un  histogramme sur une plage proche de basses magnitudes (comprise en 0 et 4, par exemple), vous pourriez également essayer d'identifier à partir de quel moment il devient intéressant de considérer les flux optiques comme signifiant un véritable mouvement. Parfois, des nombreux points ne bougent pas dans l'image, mais à cause des erreurs de mesure (ouverture, discontinuités), on leur attribue des magnitudes supérieurs à zéro.

**Q11/** Ecrivez une fonction qui filtre un flux optique en mettant à zéro les points dont la magnitude est inférieure au seuil minimal et supérieure au seuil maximal identifiés en **10**.

**Q12/** Ecrivez une fonction qui calcule un histogramme de flux optique en partant de la représentation `(orientation, magnitude)` en ignorant les points ayant une `magnitude==0`.

Pour cela vous devrez fragmenter l’espace `(orientation, magnitude)`. Si vous souhaitez disposer de `o_bins` pour fragmenter l’orientation et de `m_bins` pour fragmenter la magnitude, vous allez construire un vecteur disposant de `o_bins*m_bins` cases.

L’étendue d’une cellule sera de `o_bin_etendue = 360/o_bins` degrés pour les orientations et de `m_bin_etendue = max_mag/m_bins` pour les magnitudes.
Vous pourrez ensuite adapter la fonction color_histogram (fournie en *TP2*) pour remplir l’histogramme. 

Vous pouvez aussi choisir une implémentation moins efficace en parcourant les points composants le flux optiques et en incrémentant la case correspondante `bin_o*m_bins+bin_m`, où :
* `bin_o` correspond au bin d’orientation où le point `(o,m)` devra se trouver (`=o/o_bin_etendue`) et 
* `bin_m` correspond au bin de magnitude où le point `(o,m)` devra se trouver (`=m/m_bin_etendue`).  

**Q13/** Calculez un histogramme moyen pour une vidéo en partant des histogramme de flux optique calculés entre deux images successives et filtré comme indiqué en **Q11**.

**Q14/** Réappliquer les protocoles de classification de la **Q3 et Q5** sur ces nouveaux histogrammes moyens. Reporter et discuter les résultats.

**Q15/** Réappliquer le protocole de classification de la **Q3 et Q5** en considérant cette fois-ci uniquement des histogrammes d'orientation (en ignorant les magnitudes des flux) et en laissant également de côté les flux ayant une `magnitude == 0`. Reporter et discuter les résultats.

**Q16/** Faites varier les seuils de filtrage obtenus en **Q10** et refaites les experimentations de la **Q14** (au minium 2 autres valeurs pour le seuil haut et 2 autres valeurs pour le seuil bas). Reporter et discuter les résultats.

## Vidéos - flux optiques comme images RGB (optionnel)

**Q17/** A partir des images `RGB` illustrant les flux (voir **Q7**), vous pouvez réappliquer la même méthodologie décrite en **Q1**-**Q2** afin de calculer un descripteur global de la vidéo.

**Q18/** Ecrire une fonction qui calcule un histogramme moyen pour une vidéo, à partir des histogrammes couleur de chaque image `RGB` illustrant le flux optique entre deux images successives.

**Q19/** Si l'on considère l'histogramme moyen des images RGB des flux comme un descripteur pour une vidéo, mettez en place les protocoles de classification décrites dans les **Q3 et Q5**.

Reporter et discuter les résultats.